In [1]:
import pandas as pd
from datetime import datetime
from os import listdir, path

In [2]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None

# 1 - Adding columns of calculated data

In [9]:
#2 Checkpoint
df_formatted = df_geral.copy()

In [10]:
# Formatting dataframe for further calculations:
root_columns = ['Confirmed','Deaths','Recovered','Active']       # columns over which the calculations will be performed
sort_columns = ['Country/Region','Province/State','Date']        # order for sorting columns
MA = [3,7,15]                                                    # Moving Average intervals for computation

# pre-formatting frame:
group = ['Country/Region','Province/State','Month','Week','Day']
df_formatted = df_formatted.groupby(group).max().sort_values(by=sort_columns,axis=0)

# creating columns of daily percentage of increase in values:
for col in root_columns:
    daily_inc = col + "_daily_%inc"
    df_formatted[daily_inc] = df_formatted[col].pct_change().fillna(value=0)*100

# creating columns of moving average in root columns values:
group = ['Country/Region','Province/State']
for col in root_columns:
    for interval in MA:
        interval_col_name = col + "_" + str(interval) + "d_MA"
        df_formatted[interval_col_name] = df_formatted.groupby(group).rolling(interval)[col].mean().droplevel(level=[0,1]).fillna(method='bfill')

# Calculating the number of days since the 1st case:
df_formatted['Days_since_1st_case'] = pd.to_datetime(df_formatted.groupby(['Country/Region','Province/State'])['Date'].first())
df_formatted['Days_since_1st_case'] = (df_formatted['Date'] - df_formatted['Days_since_1st_case']).dt.days


# Calculating 1st and 2nd derivatives for root_columns:

# Since the derivative of a function is subject to noise the calculation will be performed over 
# the mean data (taking the minimum interval) in order to minimize the noise influence

interval = str(min(MA)) # The value of the minimum interval
for col in root_columns:
    column_name_first = col + '_1st_derivative'
    column_name_second = col + '_2nd_derivative'
    column_data = col + '_' + interval + "d_MA"
    df_formatted[column_name_first] = (df_formatted[column_data] - df_formatted[column_data].shift(periods=1)).fillna(method='bfill')
    df_formatted[column_name_second] = (df_formatted[column_name_first] - df_formatted[column_name_first].shift(periods=1)).fillna(method='bfill')

In [18]:
df_formatted.to_excel('formatted_data.xlsx')

# 4 - Ordering & Sorting the DataFrame columns

In [31]:
columns = ['Date', 'Country/Region', 'Province/State', 'Latitude', 'Longitude',
           'Confirmed', 'Deaths', 'Recovered', 'Active','Year', 'Month', 'Week',
           'Day','Last Update']

df_formatted = df_formatted[columns]

# 5 - Writing dataframe into external file 

In [9]:
df_formatted.size

2091565